In [1]:
import torch
import torchvision



In [2]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

/opt/anaconda3/envs/hoho_RL/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/hoho_RL/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### two-stage

目标检测分两步： 获取候选区域，然后进行分类

算法：R-CNN系列

### one-stage

一步到位。

算法：SSD、 YOLO

### 常见方法

* 滑动窗口法

暴力法

* R-CNN

通过候选区域获取感兴趣区域(ROI)：首先将每个像素作为一组。然后，计算每一组的纹理，并将两个最接近的组结合起来。继续合并区域，直到所有区域都结合在一起
![./hoho_image1.png](./hoho_image1.png)

之后使用SVM对区域进行分类，使用线性回归损失来校正边界框。

流程图如下：
![./hoho_image2.png](./hoho_image2.png)


* Fast R-CNN

使用CNN网络先提取整个图像的特征，而不是对每个图像块提取多次。然后，我们可以将创建候选区域的方法直接应用到提取到的特征图(feature map)上。


![./hoho_image3.png](./hoho_image3.png)

其中，使用ROI pooling将不同大小的候选区域输出固定大小的特征图：

![./hoho_image6.png](./hoho_image6.png)

总体流程如下：

![./hoho_image4.png](./hoho_image4.png)


* Faster R-CNN

Fast R-CNN 依赖于外部候选区域方法，如选择性搜索（跟R-CNN一样）。Faster R-CNN 采用与 Fast R-CNN 相同的设计，只是它用区域生成网络（Region Proposal Network，RPN）代替了候选区域方法，让网络自己学习自己的候选区域应该是什么。

![./hoho_image5.png](./hoho_image5.png)

### YOLO

就是一连串的卷积神经网络

![./hoho_image7.png](./hoho_image7.png)

神经网络会把448的图最后给卷积成7x7。正好就是7x7的格子。每个格子算出来就是一个点。当然有30个通道。所以每个格子其实映射出了30维的一个向量。

![./hoho_image8.png](./hoho_image8.png)

这30维向量，每个维度作用如下：

![./hoho_image9.png](./hoho_image9.png)


最后从这两个框里选出一个来。最多有7x7x2个框。丢掉置信度小的，合并同一个类别，就得到了识别结果。

### SSD

其实也是一连串卷积神经网络的堆叠，于YOLO差不多

![./hoho_image10.png](./hoho_image10.png)

SSD的框框是预设好的，各种大小的feature map上都有(多个尺度的feature map)。没有YOLO往出来算框框哪一步。由于各个尺寸上都预设了框框。所以最终框出来结果不错。